# Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Part 3

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from sklearn.cluster import KMeans
import folium 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors


print('Libraries imported.')

Libraries imported.


In [2]:

URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' 
response= requests.get(URL) 
soup = BeautifulSoup(response.text ,'html.parser')
print("done!")

done!


In [3]:
table= soup.find('table')
PostalCode   = []
Borough      = [] 
Neighbourhood = []
 
TD = table.find_all('td')
for i in range (0, len(TD), 3 ):
    PostalCode.append(TD[i].text.strip())
    Borough.append(TD[i+1].text.strip())
    Neighbourhood.append(TD[i+2].text.strip() )
    
    
Toronto = pd.DataFrame(data=[PostalCode, Borough, Neighbourhood]).transpose()
Toronto.columns = ['PostalCode', 'Borough', 'Neigbourhood']
Toronto

,PostalCode,Borough,Neigbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [4]:
Toronto['Borough'].replace('Not assigned' , np.nan, inplace= True)
Toronto.dropna( subset =['Borough'], inplace= True)
Toronto.head()

,PostalCode,Borough,Neigbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
Toronto_df = Toronto.groupby(['PostalCode','Borough'])['Neigbourhood'].apply(', '.join).reset_index()
Toronto_df.columns = ['PostalCode', 'Borough', 'Neigbourhood']
Toronto_df

,PostalCode,Borough,Neigbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [6]:
Toronto_df['Neigbourhood'].replace('Not assigned',Toronto_df ["Borough"],inplace=True) 
Toronto_df



Toronto_df.shape

(103, 3)

In [7]:
Toronto_geo = pd.read_csv('http://cocl.us/Geospatial_data')
Toronto_geo.columns = ['PostalCode', 'Latitude', 'Longitude']
Toronto_geo

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [8]:
df_tor = pd.merge(Toronto_df, Toronto_geo, on=['PostalCode'], how='inner')

df_toronto = df_tor[['Borough', 'Neigbourhood', 'PostalCode', 'Latitude', 'Longitude']].copy()

df_toronto.head()

,Borough,Neigbourhood,PostalCode,Latitude,Longitude
0,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [9]:
df_toronto

,Borough,Neigbourhood,PostalCode,Latitude,Longitude
0,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
...,...,...,...,...,...
98,York,Weston,M9N,43.706876,-79.518188
99,Etobicoke,Westmount,M9P,43.696319,-79.532242
100,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",M9R,43.688905,-79.554724
101,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",M9V,43.739416,-79.588437


In [10]:

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geopy-2.0.0          | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: d

In [11]:
! pip install geocoder
print("imported")

     |████████████████████████████████| 102kB 4.6MB/s ta 0:00:011
  Using cached https://files.pythonhosted.org/packages/d2/3d/fa76db83bf75c4f8d338c2fd15c8d33fdd7ad23a9b5e57eb6c5de26b430e/click-7.1.2-py2.py3-none-any.whl
     |████████████████████████████████| 829kB 23.5MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
imported


In [12]:
CLIENT_ID = 'QCIRJLBJOYYPE02T0Q0V35BCHXU2FQX5KYZRSL1D3D2Y5RYK' # your Foursquare ID
CLIENT_SECRET = '5NJLWP52Y0YPLHLVOW30JXINMGEHGFZ3ZAYMB0M4QJ3JYPLO' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QCIRJLBJOYYPE02T0Q0V35BCHXU2FQX5KYZRSL1D3D2Y5RYK
CLIENT_SECRET:5NJLWP52Y0YPLHLVOW30JXINMGEHGFZ3ZAYMB0M4QJ3JYPLO


In [13]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of the City of Toronto are 43.6534817, -79.3839347.


In [14]:
df_torNei = df_toronto[df_toronto['Borough'].str.contains('Toronto')]

toronto_data = df_torNei.reset_index(drop=True)
toronto_data

,Borough,Neigbourhood,PostalCode,Latitude,Longitude
0,East Toronto,The Beaches,M4E,43.676357,-79.293031
1,East Toronto,"The Danforth West, Riverdale",M4K,43.679557,-79.352188
2,East Toronto,"India Bazaar, The Beaches West",M4L,43.668999,-79.315572
3,East Toronto,Studio District,M4M,43.659526,-79.340923
4,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790
5,Central Toronto,Davisville North,M4P,43.712751,-79.390197
6,Central Toronto,"North Toronto West, Lawrence Park",M4R,43.715383,-79.405678
7,Central Toronto,Davisville,M4S,43.704324,-79.388790
8,Central Toronto,"Moore Park, Summerhill East",M4T,43.689574,-79.383160
9,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",M4V,43.686412,-79.400049


In [22]:
Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)
for lati, long, borough, neigborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neigbourhood']):
    label = '{}, {}'.format(neigborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lati, long],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.3,
        parse_html=False).add_to(Toronto_map) 
    
    Toronto_map

In [23]:
radius=500
LIMIT=100

In [24]:

toronto_data.loc[5,'Neigbourhood']

'Davisville North'

In [25]:
neighbourhood_latitude = toronto_data.loc[5, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = toronto_data.loc[5, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = toronto_data.loc[5, 'Neigbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Davisville North are 43.7127511, -79.3901975.


In [26]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

In [27]:
Finalresults = requests.get(url).json()
Finalresults


{'meta': {'code': 200, 'requestId': '5f0c0a13c7664158eb5f2c83'},
 'response': {'headerLocation': 'Davisville',
  'headerFullLocation': 'Davisville, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 7,
  'suggestedBounds': {'ne': {'lat': 43.7172511045, 'lng': -79.38398344441633},
   'sw': {'lat': 43.708251095499996, 'lng': -79.39641155558367}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8e73c30cd6209590ae7be4',
       'name': 'Summerhill Market North',
       'location': {'address': '1054 Mount Pleasant Rd.',
        'crossStreet': 'Sheldrake Blvd',
        'lat': 43.71549914910689,
        'lng': -79.39288125988016,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.71549914910689,
          'lng': -79.39288125988016}],
  

In [28]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [29]:
NEARBYvenues = Finalresults['response']['groups'][0]['items']    
NEAR_venues = json_normalize(NEARBYvenues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
NEAR_venues =NEAR_venues.loc[:, filtered_columns]

# filter the category for each row
NEAR_venues['venue.categories'] = NEAR_venues.apply(get_category_type, axis=1)

# clean columns
NEAR_venues.columns = [col.split(".")[-1] for col in NEAR_venues.columns]

NEAR_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,Summerhill Market North,Food & Drink Shop,43.715499,-79.392881
1,Sherwood Park,Park,43.716551,-79.387776
2,Homeway Restaurant & Brunch,Breakfast Spot,43.712641,-79.391557
3,Winners,Department Store,43.713236,-79.393873
4,Best Western Roehampton Hotel & Suites,Hotel,43.708878,-79.390880
